In [1]:
import numpy as np
import random
from platypus import Hypervolume, display, calculate, CMAES, MOEAD, NSGAII, NSGAIII, SPEA2, IBEA, Problem, Real
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

from datetime import datetime

In [2]:
from load_data import *
from bias_functions import *
from utility_functions import *

In [3]:
# new problems

# problem 4: age, bank
# objectives: accuracy, DI, EO, DM(OMR)

def problem4_base(w,X,y,sensitiveAttributeIndex):
    errorRateObjective = errorRate(w,X,y)
    fairnessObjective1 = differenceDisparateImpactModel(w,X,sensitiveAttributeIndex=sensitiveAttributeIndex)
    fairnessObjective2 = differenceEqualOpportunity(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex)
    fairnessObjective3 = differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex,type='OMR')
    return errorRateObjective, np.abs(fairnessObjective1[0]-fairnessObjective1[1]),\
np.abs(fairnessObjective2[0]-fairnessObjective2[1]), np.abs(fairnessObjective3[0]-fairnessObjective3[1])

def problem4_train(w):
    return problem4_base(w,trainxs,trainys,sensitiveAttributeIndex=0)
def problem4_test(w):
    return problem4_base(w,testxs,testys,sensitiveAttributeIndex=0)

In [4]:
trainAlgorithmToLoad = '20210817_1055_problem4_train_bank_CMAES_algo.npy'
testAlgorithmToLoad = '20210817_1055_problem4_test_bank_CMAES_algo.npy'

bestTrainAlgorithm = np.load("saved_algorithms/"+trainAlgorithmToLoad, allow_pickle=True).item()
bestTestAlgorithm = np.load("saved_algorithms/"+testAlgorithmToLoad, allow_pickle=True).item()

In [5]:
len(bestTrainAlgorithm.result)

1452

In [6]:
resultsMinMax(bestTrainAlgorithm)[:3]

array([[9.54172989e-02, 1.61308571e-06, 1.78468384e-06, 1.28240314e-05],
       [6.13808801e-01, 2.41697846e-01, 2.85415564e-01, 1.79112319e-01],
       [1.67295134e-01, 6.06534803e-02, 1.03103945e-01, 9.81975051e-02]])

In [14]:
# 3-D chart subject to filter

%matplotlib qt
# %matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

now = datetime.now()

axisToFilter = 0
filterMarkers = [0,0.15,0.2,0.25,1]

resultsDict = {}
resultsToUse = np.array([s.objectives[axisToFilter] for s in bestTrainAlgorithm.result])
labelNames = []

for i in range(len(filterMarkers)-1):
    resultsDict[i] = (resultsToUse>filterMarkers[i]) * (resultsToUse<=filterMarkers[i+1])
    if filterMarkers[i] == 0:
        labelName = '{:,.0%} < acc'.format(1-filterMarkers[i+1])
    elif filterMarkers[i+1] == 1:
        labelName = 'acc < {:,.0%}'.format(1-filterMarkers[i])
    else:
        labelName = '{:,.0%} < acc < {:,.0%}'.format(1-filterMarkers[i+1],1-filterMarkers[i])
    labelNames.append(labelName)

fig = plt.figure(dpi=200)
ax = fig.add_subplot(111, projection='3d')

for i in range(len(filterMarkers)-1):
    ax.scatter(np.array([s.objectives[1] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[2] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[3] for s in bestTrainAlgorithm.result])[resultsDict[i]],s=1)

ax.set_xlabel("Disparate Impact",labelpad=10,size=6)
ax.set_ylabel("Equal Opportunity",labelpad=10,size=6)
ax.set_zlabel("Disparate Mistreatment (OMR)",labelpad=10,size=6)
ax.tick_params(axis='x', labelsize=6)
ax.tick_params(axis='y', labelsize=6)
ax.tick_params(axis='z', labelsize=6)
ax.set_xlim([0,0.25])
ax.set_ylim([0,0.3])
ax.set_zlim([0,0.2])
ax.view_init(elev=15, azim=-105)
ax.legend(labelNames,fontsize=5) 

# plt.savefig("saved_graphs/"+now.strftime("%Y%m%d_%H%M")+"_"+dataset+"_"+str(bestTrainAlgorithm)[21:-26]+"_axis"+str(axisToFilter)+"_overall",bbox_inches = 'tight')

In [ ]:
# 3-D chart subject to filter

%matplotlib qt
# %matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

now = datetime.now()

axisToFilter = 1
filterMarkers = [0,0.05,0.1,0.15,1]

resultsDict = {}
resultsToUse = np.array([s.objectives[axisToFilter] for s in bestTrainAlgorithm.result])
labelNames = []

for i in range(len(filterMarkers)-1):
    resultsDict[i] = (resultsToUse>=filterMarkers[i]) * (resultsToUse<filterMarkers[i+1])
    if filterMarkers[i] == 0:
        labelName = 'DI < {:,.2f}'.format(filterMarkers[i+1])
    elif filterMarkers[i+1] == 1:
        labelName = '{:,.2f} < DI'.format(filterMarkers[i])
    else:
        labelName = '{:,.2f} < DI < {:,.2f}'.format(filterMarkers[i],filterMarkers[i+1])
    labelNames.append(labelName)

fig = plt.figure(dpi=200)
ax = fig.add_subplot(111, projection='3d')

for i in range(len(filterMarkers)-1):
    ax.scatter(np.array([s.objectives[0] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[2] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[3] for s in bestTrainAlgorithm.result])[resultsDict[i]],s=1)

ax.set_xlabel("Error Rate",labelpad=10,size=6)
ax.set_ylabel("Equal Opportunity",labelpad=10,size=6)
ax.set_zlabel("Disparate Mistreatment (OMR)",labelpad=10,size=6)
ax.tick_params(axis='x', labelsize=6)
ax.tick_params(axis='y', labelsize=6)
ax.tick_params(axis='z', labelsize=6)
ax.set_xlim([0,0.65])
ax.set_ylim([0,0.3])
ax.set_zlim([0,0.2])
ax.view_init(elev=15, azim=-125)
ax.legend(labelNames,fontsize=5) 

# plt.savefig("saved_graphs/"+now.strftime("%Y%m%d_%H%M")+"_"+dataset+"_"+str(bestTrainAlgorithm)[21:-26]+"_axis"+str(axisToFilter)+"_overall",bbox_inches = 'tight')

In [13]:
# 3-D chart subject to filter

%matplotlib qt
# %matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

now = datetime.now()

axisToFilter = 2
filterMarkers = [0,0.1,0.2,1]

resultsDict = {}
resultsToUse = np.array([s.objectives[axisToFilter] for s in bestTrainAlgorithm.result])
labelNames = []

for i in range(len(filterMarkers)-1):
    resultsDict[i] = (resultsToUse>=filterMarkers[i]) * (resultsToUse<filterMarkers[i+1])
    if filterMarkers[i] == 0:
        labelName = 'EO < {:,.2f}'.format(filterMarkers[i+1])
    elif filterMarkers[i+1] == 1:
        labelName = '{:,.2f} < EO'.format(filterMarkers[i])
    else:
        labelName = '{:,.2f} < EO < {:,.2f}'.format(filterMarkers[i],filterMarkers[i+1])
    labelNames.append(labelName)

fig = plt.figure(dpi=200)
ax = fig.add_subplot(111, projection='3d')

for i in range(len(filterMarkers)-1):
    ax.scatter(np.array([s.objectives[0] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[1] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[3] for s in bestTrainAlgorithm.result])[resultsDict[i]],s=1)

ax.set_xlabel("Error Rate",labelpad=10,size=6)
ax.set_ylabel("Disparate Impact",labelpad=10,size=6)
ax.set_zlabel("Disparate Mistreatment (OMR)",labelpad=10,size=6)
ax.tick_params(axis='x', labelsize=5)
ax.tick_params(axis='y', labelsize=5)
ax.tick_params(axis='z', labelsize=5)
ax.set_xlim([0,0.4])
ax.set_ylim([0,0.2])
ax.set_zlim([0,0.2])
ax.view_init(elev=15, azim=-125)
ax.legend(labelNames,fontsize=5) 

# plt.savefig("saved_graphs/"+now.strftime("%Y%m%d_%H%M")+"_"+dataset+"_"+str(bestTrainAlgorithm)[21:-26]+"_axis"+str(axisToFilter)+"_overall",bbox_inches = 'tight')

In [15]:
# 3-D chart subject to filter

%matplotlib qt
# %matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

now = datetime.now()

axisToFilter = 3
filterMarkers = [0,0.05,0.1,0.15,1]

resultsDict = {}
resultsToUse = np.array([s.objectives[axisToFilter] for s in bestTrainAlgorithm.result])
labelNames = []

for i in range(len(filterMarkers)-1):
    resultsDict[i] = (resultsToUse>=filterMarkers[i]) * (resultsToUse<filterMarkers[i+1])
    if filterMarkers[i] == 0:
        labelName = 'DM (OMR) < {:,.2f}'.format(filterMarkers[i+1])
    elif filterMarkers[i+1] == 1:
        labelName = '{:,.2f} < DM (OMR)'.format(filterMarkers[i])
    else:
        labelName = '{:,.2f} < DM (OMR) < {:,.2f}'.format(filterMarkers[i],filterMarkers[i+1])
    labelNames.append(labelName)

fig = plt.figure(dpi=200)
ax = fig.add_subplot(111, projection='3d')

for i in range(len(filterMarkers)-1):
    ax.scatter(np.array([s.objectives[0] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[1] for s in bestTrainAlgorithm.result])[resultsDict[i]],
           np.array([s.objectives[2] for s in bestTrainAlgorithm.result])[resultsDict[i]],s=1)

ax.set_xlabel("Error Rate",labelpad=10,size=6)
ax.set_ylabel("Disparate Impact",labelpad=10,size=6)
ax.set_zlabel("Equal Opportunity",labelpad=10,size=6)
ax.tick_params(axis='x', labelsize=6)
ax.tick_params(axis='y', labelsize=6)
ax.tick_params(axis='z', labelsize=6)
ax.set_xlim([0,0.4])
ax.set_ylim([0,0.2])
ax.set_zlim([0,0.3])
ax.view_init(elev=15, azim=-125)
ax.legend(labelNames,fontsize=5) 

# plt.savefig("saved_graphs/"+now.strftime("%Y%m%d_%H%M")+"_"+dataset+"_"+str(bestTrainAlgorithm)[21:-26]+"_axis"+str(axisToFilter)+"_overall",bbox_inches = 'tight')